In [1]:
!pip install torch

In [2]:
!pip install idx2numpy

In [3]:
!pip install sklearn

In [4]:
!pip install pyreadr


In [59]:
# Define random state
random_state = 0

# Import libraries, modules, and functions
import time
import numpy as np
import matplotlib.pyplot as plt

# PyTorch imports
import torch
import torch.nn as nn
import idx2numpy
from sklearn.model_selection import KFold

In [60]:
# Display GPU/CUDA availability
print('\nCUDA availability:  %s\n' % (torch.cuda.is_available()))



CUDA availability:  False



In [61]:
data_path='/Users/ali/Desktop/Jul/apoe2_paper/vertex/age_cat/' # data path after downloading from mnist website abnd unziping in a folder

In [62]:
import pyreadr
image=pyreadr.read_r(data_path + 'image_filter.rda')
#print(image.keys())
image = image["image_filter"]
#print(image.shape)
age_cat=pyreadr.read_r(data_path + 'response.rda')
#print(sex.keys())
age_cat= age_cat["response"]
#print(sex.shape)
#print(sex)
age_cat=age_cat["age_cat"]
print(age_cat)

0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    1.0
12    1.0
13    1.0
14    1.0
15    1.0
16    1.0
17    1.0
18    1.0
19    1.0
20    1.0
21    0.0
22    1.0
23    1.0
24    1.0
25    1.0
26    1.0
27    1.0
28    1.0
29    1.0
30    1.0
31    1.0
32    0.0
Name: age_cat, dtype: float64


In [63]:
age_cat[age_cat==0]=0;
age_cat[age_cat==1]=1;
type(age_cat)
age_cat

0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    1.0
12    1.0
13    1.0
14    1.0
15    1.0
16    1.0
17    1.0
18    1.0
19    1.0
20    1.0
21    0.0
22    1.0
23    1.0
24    1.0
25    1.0
26    1.0
27    1.0
28    1.0
29    1.0
30    1.0
31    1.0
32    0.0
Name: age_cat, dtype: float64

In [64]:
X_raw_train_npy = image #
y_train_npy = age_cat

In [65]:
#flatten images
X_raw_train_npy =  np.reshape(X_raw_train_npy, newshape =(X_raw_train_npy.shape[0],-1))

In [66]:
print(X_raw_train_npy.shape)
print(X_raw_train_npy.shape[0])
print(y_train_npy.shape)


(33, 77)
33
(33,)


### store original data

In [67]:
X_raw_train_npy=X_raw_train_npy.to_numpy()
y_train_npy=y_train_npy.to_numpy()

In [68]:
X_raw_train_npy_orig =  X_raw_train_npy
y_train_npy_orig = y_train_npy

In [69]:
y_train_npy = np.array(y_train_npy, dtype=int)
y_train_npy

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0])

In [70]:
y_train_npy_orig.shape 

type(y_train_npy_orig)

numpy.ndarray

In [71]:
nfold=5
kfold = KFold(nfold,  True )
for train, test in kfold.split(X_raw_train_npy_orig):
    print( type(X_raw_train_npy_orig[test].shape))
    

<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>


In [72]:
nfold=5
kfold = KFold(nfold,  True )
for train, test in kfold.split(X_raw_train_npy_orig):
	print('train: %s, test: %s' % (X_raw_train_npy_orig[train].shape, X_raw_train_npy_orig[test].shape))
	print('train: %s, test: %s' % (y_train_npy_orig[train].shape, y_train_npy_orig[test].shape))

train: (26, 77), test: (7, 77)
train: (26,), test: (7,)
train: (26, 77), test: (7, 77)
train: (26,), test: (7,)
train: (26, 77), test: (7, 77)
train: (26,), test: (7,)
train: (27, 77), test: (6, 77)
train: (27,), test: (6,)
train: (27, 77), test: (6, 77)
train: (27,), test: (6,)


In [73]:
y_train_npy_orig[train]

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 0.])

In [74]:
# MNIST PyTorch dataset class definition
class MNISTDataset(torch.utils.data.Dataset):
    """
    PyTorch dataset object for the MNIST dataset.
    
    Parameters
    ----------
    X : torch.Tensor
        Flattened image data of size (n_samples x 784).
    y : torch.Tensor
        Image labels, integer values 0, 1, 2, ..., 9.
    
    Attributes
    ----------
    X : torch.Tensor
        Flattened image data of size (n_samples x 784).
    y : torch.Tensor
        Image labels, integer values 0, 1, 2, ..., 9.
    len : int
        Length of dataset/number of observations in dataset.
    """
    
    # MNIST PyTorch dataset instantiation method
    def __init__(self, X, y):
        """
        MNIST dataset instantiation method.
        
        Parameters
        ----------
        X : torch.Tensor
            Flattened image data of size (n_samples x 784).
        y : torch.Tensor
            Image labels, integer values 0, 1, 2, ..., 9.
        
        Attributes
        ----------
        X : torch.Tensor
            Flattened image data of size (n_samples x 784).
        y : torch.Tensor
            Image labels, integer values 0, 1, 2, ..., 9.
        len : int
            Length of dataset/number of observations in dataset.
        """
        
        # Assign attributes
        self.X = X         # flattened image data
        self.y = y         # image labels
        self.len = len(X)  # length of dataset

    # Dataset length method
    def __len__(self):
        """
        Method that returns length of dataset/number of observations in dataset.
        
        Returns
        -------
        self.len : int
            Length of dataset/number of observations in dataset.
        """
        
        return self.len

    # Dataset indexing method
    def __getitem__(self, idx):
        """
        Method that retrieves samples and corresponding labels from the dataset.
        
        Parameters
        ----------
        idx : int
            Integer index of dataset sample to be retrieved.
        
        Returns
        -------
        X_samp : torch.Tensor
            Image sample.
        y_samp : torch.Tensor
            Label corresponding to image sample.
        """
        
        # Retrieve sample of data indexed by idx
        X_samp = self.X[idx, :]
        y_samp = self.y[idx]

        # Return data sample
        return X_samp, y_samp


In [75]:
# Logistic regression model class definition
class LogisticRegression(nn.Module):
    """
    PyTorch implementation of a logistic regression model.
    
    Parameters
    ----------
    input_dim : int
        Size/dimensionality of the model input data.
    output_dim : int
        Size/dimensionality of the model output.

    Attributes
    ----------
    input_dim : int
        Size/dimensionality of the model input data.
    output_dim : int
        Size/dimensionality of the model output.
    linear : torch.nn.Linear
        Fully-connected/linear layer.
    """
    
    # Logistic regression instantiation method
    def __init__(self, input_dim, output_dim):
        """
        Logistic regression model instantiation method.
        
        Parameters
        ----------
        input_dim : int
            Size/dimensionality of the model input data.
        output_dim : int
            Size/dimensionality of the model output.

        Attributes
        ----------
        input_dim : int
            Size/dimensionality of the model input data.
        output_dim : int
            Size/dimensionality of the model output.
        linear : torch.nn.Linear
            Fully-connected/linear layer.
        """
        
        # Inherit from torch.nn.Module
        super(LogisticRegression, self).__init__()
        
        # Assign logistic regression parameters to model attributes
        self.input_dim = input_dim
        self.output_dim = output_dim
        
        self.linear = nn.Linear(
            in_features=input_dim,
            out_features=output_dim,
            bias=True)
    
    # Logistic regression forward pass method
    def forward(self, x):
        """
        Logistic regression forward pass method.
        
        Parameters
        ----------
        x : torch.Tensor()
            Tensor of input data.
        
        Returns
        -------
        logits : torch.Tensor()
             Raw model predictions (not passed through sigmoid or softmax function).
        """
        
        # Calculate model predictions/logits
        logits = self.linear(x)
        
        # Return model output
        return logits


In [76]:
# Instantiate cross-entropy loss function
loss_func = nn.CrossEntropyLoss()


In [77]:
# Number of training epochs (epochs = # of passes through data)
n_epochs = 500

In [78]:
import random
torch.manual_seed(0)
np.random.seed(0)

In [79]:

testkfold=[]
nfold=5
kfold = KFold(nfold,  True )
for train, test in kfold.split(X_raw_train_npy_orig):
#	print('train: %s, test: %s' % (X_raw_train_npy_orig[train].shape, X_raw_train_npy_orig[test].shape))
    X_raw_train=X_raw_train_npy_orig[train]
    X_raw_test=X_raw_train_npy_orig[test]
    y_train=np.array(y_train_npy_orig[train] , dtype=float)
    y_test=np.array(y_train_npy_orig[test], dtype=float)
    #y_train = torch.tensor(y_train, dtype=torch.long)
    #y_test = torch.tensor(y_test, dtype=torch.long)


    
    
    X_raw_train = torch.from_numpy(X_raw_train) #format torch data
    y_train = torch.from_numpy(y_train)
    X_raw_test = torch.from_numpy(X_raw_test)
    y_test = torch.from_numpy(y_test)

    X_raw_train = X_raw_train.float() #float
    X_raw_test = X_raw_test.float()
    
    #X_train = X_raw_train / 255.0 #scale
    #X_test = X_raw_test / 255.0
    X_train=X_raw_train
    X_test=X_raw_test
    
    
    train_dataset = MNISTDataset(X_train, y_train) #pytorch data
    test_dataset = MNISTDataset(X_test, y_test)
    
    batch_size = 10
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    input_dim = X_train.shape[1]             #Define model input and output dimensions from data
    output_dim = len(np.unique(y_train.numpy()))  

    
    lr_model = LogisticRegression(input_dim=input_dim, output_dim=output_dim)# Instantiate logistic regression model
    
    lr = 0.1# Instantiate stochastic gradient descent (SGD) optimizer
    
    optimizer = torch.optim.SGD(lr_model.parameters(), lr=lr)
    #optimizer = torch.optim.Adam(lr_model.parameters())
    
    print('\nTraining logistic regression model...\n')# Time model training
    time_start = time.time()
    
    lr_model.train()# train() method affects operations such as dropout and batch normalization
        # Train model/iterate over epochs
    for epoch in range(n_epochs):
        # Initialize epoch metrics variables
        n_obs = 0
        loss_sum = 0
        n_correct = 0

        # Iterate through training data mini-batchesbatches
        for images_batch, labels_batch in train_loader:
            # Zero out the gradients
            optimizer.zero_grad()

            # Forward pass
            y_pred = lr_model(images_batch)         # model predictions
            loss = loss_func(y_pred, labels_batch.long())  # loss function evaluation

            # Backward pass
            loss.backward()    # backpropagation
            optimizer.step()   # update parameters according to learning rate, gradients

            # Update epoch metrics variables
            n_batch = len(labels_batch)
            n_obs += n_batch
            loss_sum += n_batch * loss.item()
            n_correct += (y_pred.argmax(1) == labels_batch.long()).float().sum().item()

        # Calculate epoch training loss and training accuracy
        loss_train = loss_sum / n_obs
        acc_train = n_correct / n_obs

        # Display training progress
        prog_disp_freq = 1   # frequency of training progress display
        if (epoch + 1) % prog_disp_freq == 0 or epoch == 0 or (epoch + 1) == n_epochs:
            print('  E%02d | train loss: %s | train acc.: %s' % 
                  (epoch + 1, '{:.4f}'.format(loss_train), '{:.4f}'.format(acc_train)))

    # Time model training
    time_end = time.time()
    print('\nLogistic regression model training complete.\n')
    print('Time to train logistic regression model:  %.1f s\n' % (time_end - time_start))
        # Place model in evaluation mode
    # .eval() method affects operations such as dropout and batch normalization
    lr_model.eval()

    # Initialize test set metrics variables
    n_obs = 0
    n_correct = 0

    # Disable gradient calculation
    with torch.no_grad():
        # Iterate through test data mini-batches
        for images_batch, labels_batch in test_loader:
            # Forward pass
            y_pred = lr_model(images_batch)  # model predictions

            # Update test set metrics variables
            n_batch = len(labels_batch)
            n_obs += n_batch
            n_correct += (y_pred.argmax(1) == labels_batch.long()).float().sum().item()

    # Calculate test accuracy
    acc_test = n_correct / n_obs

    # Display test accuracy
    print('\nLogistic regression model MNIST test acc.:  %.4f\n' % (acc_test))
    testkfold.append(acc_test)



Training logistic regression model...

  E01 | train loss: 0.6816 | train acc.: 0.4615
  E02 | train loss: 0.2847 | train acc.: 0.9231
  E03 | train loss: 0.1926 | train acc.: 1.0000
  E04 | train loss: 0.1531 | train acc.: 1.0000
  E05 | train loss: 0.1240 | train acc.: 1.0000
  E06 | train loss: 0.1039 | train acc.: 1.0000
  E07 | train loss: 0.0916 | train acc.: 1.0000
  E08 | train loss: 0.0800 | train acc.: 1.0000
  E09 | train loss: 0.0717 | train acc.: 1.0000
  E10 | train loss: 0.0653 | train acc.: 1.0000
  E11 | train loss: 0.0601 | train acc.: 1.0000
  E12 | train loss: 0.0553 | train acc.: 1.0000
  E13 | train loss: 0.0501 | train acc.: 1.0000
  E14 | train loss: 0.0469 | train acc.: 1.0000
  E15 | train loss: 0.0435 | train acc.: 1.0000
  E16 | train loss: 0.0411 | train acc.: 1.0000
  E17 | train loss: 0.0386 | train acc.: 1.0000
  E18 | train loss: 0.0363 | train acc.: 1.0000
  E19 | train loss: 0.0344 | train acc.: 1.0000
  E20 | train loss: 0.0327 | train acc.: 1.0000


  E201 | train loss: 0.0032 | train acc.: 1.0000
  E202 | train loss: 0.0032 | train acc.: 1.0000
  E203 | train loss: 0.0032 | train acc.: 1.0000
  E204 | train loss: 0.0032 | train acc.: 1.0000
  E205 | train loss: 0.0032 | train acc.: 1.0000
  E206 | train loss: 0.0032 | train acc.: 1.0000
  E207 | train loss: 0.0031 | train acc.: 1.0000
  E208 | train loss: 0.0031 | train acc.: 1.0000
  E209 | train loss: 0.0031 | train acc.: 1.0000
  E210 | train loss: 0.0031 | train acc.: 1.0000
  E211 | train loss: 0.0031 | train acc.: 1.0000
  E212 | train loss: 0.0031 | train acc.: 1.0000
  E213 | train loss: 0.0031 | train acc.: 1.0000
  E214 | train loss: 0.0030 | train acc.: 1.0000
  E215 | train loss: 0.0030 | train acc.: 1.0000
  E216 | train loss: 0.0030 | train acc.: 1.0000
  E217 | train loss: 0.0030 | train acc.: 1.0000
  E218 | train loss: 0.0030 | train acc.: 1.0000
  E219 | train loss: 0.0030 | train acc.: 1.0000
  E220 | train loss: 0.0030 | train acc.: 1.0000
  E221 | train loss:

  E407 | train loss: 0.0016 | train acc.: 1.0000
  E408 | train loss: 0.0016 | train acc.: 1.0000
  E409 | train loss: 0.0016 | train acc.: 1.0000
  E410 | train loss: 0.0016 | train acc.: 1.0000
  E411 | train loss: 0.0016 | train acc.: 1.0000
  E412 | train loss: 0.0016 | train acc.: 1.0000
  E413 | train loss: 0.0016 | train acc.: 1.0000
  E414 | train loss: 0.0016 | train acc.: 1.0000
  E415 | train loss: 0.0016 | train acc.: 1.0000
  E416 | train loss: 0.0016 | train acc.: 1.0000
  E417 | train loss: 0.0016 | train acc.: 1.0000
  E418 | train loss: 0.0016 | train acc.: 1.0000
  E419 | train loss: 0.0015 | train acc.: 1.0000
  E420 | train loss: 0.0015 | train acc.: 1.0000
  E421 | train loss: 0.0015 | train acc.: 1.0000
  E422 | train loss: 0.0015 | train acc.: 1.0000
  E423 | train loss: 0.0015 | train acc.: 1.0000
  E424 | train loss: 0.0015 | train acc.: 1.0000
  E425 | train loss: 0.0015 | train acc.: 1.0000
  E426 | train loss: 0.0015 | train acc.: 1.0000
  E427 | train loss:

  E110 | train loss: 0.0058 | train acc.: 1.0000
  E111 | train loss: 0.0057 | train acc.: 1.0000
  E112 | train loss: 0.0057 | train acc.: 1.0000
  E113 | train loss: 0.0056 | train acc.: 1.0000
  E114 | train loss: 0.0056 | train acc.: 1.0000
  E115 | train loss: 0.0055 | train acc.: 1.0000
  E116 | train loss: 0.0055 | train acc.: 1.0000
  E117 | train loss: 0.0054 | train acc.: 1.0000
  E118 | train loss: 0.0054 | train acc.: 1.0000
  E119 | train loss: 0.0054 | train acc.: 1.0000
  E120 | train loss: 0.0053 | train acc.: 1.0000
  E121 | train loss: 0.0053 | train acc.: 1.0000
  E122 | train loss: 0.0052 | train acc.: 1.0000
  E123 | train loss: 0.0052 | train acc.: 1.0000
  E124 | train loss: 0.0051 | train acc.: 1.0000
  E125 | train loss: 0.0051 | train acc.: 1.0000
  E126 | train loss: 0.0051 | train acc.: 1.0000
  E127 | train loss: 0.0050 | train acc.: 1.0000
  E128 | train loss: 0.0050 | train acc.: 1.0000
  E129 | train loss: 0.0049 | train acc.: 1.0000
  E130 | train loss:

  E319 | train loss: 0.0020 | train acc.: 1.0000
  E320 | train loss: 0.0020 | train acc.: 1.0000
  E321 | train loss: 0.0020 | train acc.: 1.0000
  E322 | train loss: 0.0020 | train acc.: 1.0000
  E323 | train loss: 0.0020 | train acc.: 1.0000
  E324 | train loss: 0.0020 | train acc.: 1.0000
  E325 | train loss: 0.0020 | train acc.: 1.0000
  E326 | train loss: 0.0019 | train acc.: 1.0000
  E327 | train loss: 0.0019 | train acc.: 1.0000
  E328 | train loss: 0.0019 | train acc.: 1.0000
  E329 | train loss: 0.0019 | train acc.: 1.0000
  E330 | train loss: 0.0019 | train acc.: 1.0000
  E331 | train loss: 0.0019 | train acc.: 1.0000
  E332 | train loss: 0.0019 | train acc.: 1.0000
  E333 | train loss: 0.0019 | train acc.: 1.0000
  E334 | train loss: 0.0019 | train acc.: 1.0000
  E335 | train loss: 0.0019 | train acc.: 1.0000
  E336 | train loss: 0.0019 | train acc.: 1.0000
  E337 | train loss: 0.0019 | train acc.: 1.0000
  E338 | train loss: 0.0019 | train acc.: 1.0000
  E339 | train loss:

...

  E131 | train loss: 0.0050 | train acc.: 1.0000
  E132 | train loss: 0.0050 | train acc.: 1.0000
  E133 | train loss: 0.0049 | train acc.: 1.0000
  E134 | train loss: 0.0049 | train acc.: 1.0000
  E135 | train loss: 0.0049 | train acc.: 1.0000
  E136 | train loss: 0.0048 | train acc.: 1.0000
  E137 | train loss: 0.0048 | train acc.: 1.0000
  E138 | train loss: 0.0048 | train acc.: 1.0000
  E139 | train loss: 0.0047 | train acc.: 1.0000
  E140 | train loss: 0.0047 | train acc.: 1.0000
  E141 | train loss: 0.0046 | train acc.: 1.0000
  E142 | train loss: 0.0046 | train acc.: 1.0000
  E143 | train loss: 0.0046 | train acc.: 1.0000
  E144 | train loss: 0.0046 | train acc.: 1.0000
  E145 | train loss: 0.0045 | train acc.: 1.0000
  E146 | train loss: 0.0045 | train acc.: 1.0000
  E147 | train loss: 0.0045 | train acc.: 1.0000
  E148 | train loss: 0.0044 | train acc.: 1.0000
  E149 | train loss: 0.0044 | train acc.: 1.0000
  E150 | train loss: 0.0044 | train acc.: 1.0000
  E151 | train loss:

  E329 | train loss: 0.0020 | train acc.: 1.0000
  E330 | train loss: 0.0020 | train acc.: 1.0000
  E331 | train loss: 0.0020 | train acc.: 1.0000
  E332 | train loss: 0.0020 | train acc.: 1.0000
  E333 | train loss: 0.0020 | train acc.: 1.0000
  E334 | train loss: 0.0020 | train acc.: 1.0000
  E335 | train loss: 0.0020 | train acc.: 1.0000
  E336 | train loss: 0.0020 | train acc.: 1.0000
  E337 | train loss: 0.0019 | train acc.: 1.0000
  E338 | train loss: 0.0019 | train acc.: 1.0000
  E339 | train loss: 0.0019 | train acc.: 1.0000
  E340 | train loss: 0.0019 | train acc.: 1.0000
  E341 | train loss: 0.0019 | train acc.: 1.0000
  E342 | train loss: 0.0019 | train acc.: 1.0000
  E343 | train loss: 0.0019 | train acc.: 1.0000
  E344 | train loss: 0.0019 | train acc.: 1.0000
  E345 | train loss: 0.0019 | train acc.: 1.0000
  E346 | train loss: 0.0019 | train acc.: 1.0000
  E347 | train loss: 0.0019 | train acc.: 1.0000
  E348 | train loss: 0.0019 | train acc.: 1.0000
  E349 | train loss:

  E20 | train loss: 0.0360 | train acc.: 1.0000
  E21 | train loss: 0.0344 | train acc.: 1.0000
  E22 | train loss: 0.0331 | train acc.: 1.0000
  E23 | train loss: 0.0315 | train acc.: 1.0000
  E24 | train loss: 0.0302 | train acc.: 1.0000
  E25 | train loss: 0.0289 | train acc.: 1.0000
  E26 | train loss: 0.0278 | train acc.: 1.0000
  E27 | train loss: 0.0267 | train acc.: 1.0000
  E28 | train loss: 0.0258 | train acc.: 1.0000
  E29 | train loss: 0.0249 | train acc.: 1.0000
  E30 | train loss: 0.0240 | train acc.: 1.0000
  E31 | train loss: 0.0233 | train acc.: 1.0000
  E32 | train loss: 0.0225 | train acc.: 1.0000
  E33 | train loss: 0.0218 | train acc.: 1.0000
  E34 | train loss: 0.0212 | train acc.: 1.0000
  E35 | train loss: 0.0206 | train acc.: 1.0000
  E36 | train loss: 0.0200 | train acc.: 1.0000
  E37 | train loss: 0.0195 | train acc.: 1.0000
  E38 | train loss: 0.0189 | train acc.: 1.0000
  E39 | train loss: 0.0185 | train acc.: 1.0000
  E40 | train loss: 0.0180 | train acc.:

  E222 | train loss: 0.0032 | train acc.: 1.0000
  E223 | train loss: 0.0032 | train acc.: 1.0000
  E224 | train loss: 0.0032 | train acc.: 1.0000
  E225 | train loss: 0.0032 | train acc.: 1.0000
  E226 | train loss: 0.0032 | train acc.: 1.0000
  E227 | train loss: 0.0031 | train acc.: 1.0000
  E228 | train loss: 0.0031 | train acc.: 1.0000
  E229 | train loss: 0.0031 | train acc.: 1.0000
  E230 | train loss: 0.0031 | train acc.: 1.0000
  E231 | train loss: 0.0031 | train acc.: 1.0000
  E232 | train loss: 0.0031 | train acc.: 1.0000
  E233 | train loss: 0.0031 | train acc.: 1.0000
  E234 | train loss: 0.0031 | train acc.: 1.0000
  E235 | train loss: 0.0030 | train acc.: 1.0000
  E236 | train loss: 0.0030 | train acc.: 1.0000
  E237 | train loss: 0.0030 | train acc.: 1.0000
  E238 | train loss: 0.0030 | train acc.: 1.0000
  E239 | train loss: 0.0030 | train acc.: 1.0000
  E240 | train loss: 0.0030 | train acc.: 1.0000
  E241 | train loss: 0.0030 | train acc.: 1.0000
  E242 | train loss:

  E421 | train loss: 0.0017 | train acc.: 1.0000
  E422 | train loss: 0.0017 | train acc.: 1.0000
  E423 | train loss: 0.0017 | train acc.: 1.0000
  E424 | train loss: 0.0017 | train acc.: 1.0000
  E425 | train loss: 0.0017 | train acc.: 1.0000
  E426 | train loss: 0.0017 | train acc.: 1.0000
  E427 | train loss: 0.0017 | train acc.: 1.0000
  E428 | train loss: 0.0017 | train acc.: 1.0000
  E429 | train loss: 0.0017 | train acc.: 1.0000
  E430 | train loss: 0.0017 | train acc.: 1.0000
  E431 | train loss: 0.0017 | train acc.: 1.0000
  E432 | train loss: 0.0016 | train acc.: 1.0000
  E433 | train loss: 0.0016 | train acc.: 1.0000
  E434 | train loss: 0.0016 | train acc.: 1.0000
  E435 | train loss: 0.0016 | train acc.: 1.0000
  E436 | train loss: 0.0016 | train acc.: 1.0000
  E437 | train loss: 0.0016 | train acc.: 1.0000
  E438 | train loss: 0.0016 | train acc.: 1.0000
  E439 | train loss: 0.0016 | train acc.: 1.0000
  E440 | train loss: 0.0016 | train acc.: 1.0000
  E441 | train loss:

In [80]:
np.mean(testkfold)

0.738095238095238

In [81]:
testkfold

[0.5714285714285714,
 0.7142857142857143,
 0.5714285714285714,
 0.8333333333333334,
 1.0]

In [82]:
# Multi-layer perceptron model class definition
class MLP(nn.Module):
    """
    PyTorch implementation of a multi-layer perceptron model.
    
    Parameters
    ----------
    input_dim : int
        Size/dimensionality of the model input data.
    output_dim : int
        Size/dimensionality of the model output.
    hidden_dim : int
        Size/dimensionality of the hidden layer.
    
    Attributes
    ----------
    input_dim : int
        Size/dimensionality of the model input data.
    output_dim : int
        Size/dimensionality of the model output.
    hidden_dim : int
        Size/dimensionality of the hidden layer.
    linear_1 : torch.nn.Linear
        Fully-connected/linear neural network layer.
    linear_2 : torch.nn.Linear
        Fully-connected/linear neural network layer.
    relu : torch.nn.ReLU
        Rectified linear unit activation function.
    """
    
    # MLP instantiation method
    def __init__(self, input_dim, output_dim, hidden_dim=15):
        """
        Model instantiation method.
        
        Parameters
        ----------
        input_dim : int
            Size/dimensionality of the model input data.
        output_dim : int
            Size/dimensionality of the model output.
        hidden_dim : int
            Size/dimensionality of the hidden layer.

        Attributes
        ----------
        input_dim : int
            Size/dimensionality of the model input data.
        output_dim : int
            Size/dimensionality of the model output.
        hidden_dim : int
            Size/dimensionality of the hidden layer.
        linear_1 : torch.nn.Linear
            Fully-connected/linear neural network layer.
        linear_2 : torch.nn.Linear
            Fully-connected/linear neural network layer.
        relu : torch.nn.ReLU
            Rectified linear unit activation function.
        """
        
        # Inherit from torch.nn.Module
        super(MLP, self).__init__()
        
        # Assign MLP parameters to model attributes
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        
        # Define MLP layers
        self.linear_1 = nn.Linear(
            in_features=input_dim,
            out_features=hidden_dim,
            bias=True)
        self.linear_2 = nn.Linear(
            in_features=hidden_dim,
            out_features=hidden_dim,
            bias=True)
        self.linear_3 = nn.Linear(
            in_features=hidden_dim,
            out_features=output_dim,
            bias=True)
        
        # Define MLP activation function
        self.relu = nn.LeakyReLU()
    
    # MLP forward pass method
    def forward(self, x):
        """
        MLP forward pass method.
        
        Parameters
        ----------
        x : torch.Tensor()
            Tensor of input data.
        
        Returns
        -------
        logits : torch.Tensor()
             Raw model predictions (not passed through sigmoid or softmax function).
        """
        
        # Forward pass through model
        x = self.linear_1(x)       # first fully-connected layer
        x = self.relu(x)           # non-linear activation transformation
        
        s=self.linear_2(x)
        s = self.relu(s)           # non-linear activation transformation

        mids2=self.linear_2(s)
        mids2 = self.relu(mids2)           # non-linear activation transformation
        
        mids3=self.linear_2(mids2)
        mids3 = self.relu(mids3)           # non-linear activation transformation

        
        mids4=self.linear_2(mids3)
        mids4 = self.relu(mids4)
        
        mids5=self.linear_2(mids4)
        mids5 = self.relu(mids5)
        
        
        logits = self.linear_3(s)  # second fully-connected layer
        
        # Return model output
        return logits


In [83]:
X_raw_train_npy_orig

array([[ 0.87289496, -0.04490834,  0.08906234, ..., -0.17407766,
        -0.81636418, -0.34304762],
       [ 0.08370226, -0.32990355,  2.11599841, ..., -0.17407766,
        -0.81636418,  1.92106668],
       [-0.3108941 , -0.32990355, -0.21497807, ..., -0.17407766,
        -0.02401071, -0.34304762],
       ...,
       [-0.11359592, -0.27290451, -0.41767167, ..., -0.17407766,
        -0.02401071, -0.34304762],
       [-0.70549045, -0.44390164,  0.79848997, ..., -0.17407766,
         1.56069623, -0.34304762],
       [ 3.8323676 ,  3.03303998,  0.08906234, ..., -0.17407766,
         2.35304969,  1.92106668]])

In [84]:
import random
random.seed(1)
torch.manual_seed(0)

In [85]:
testkfold=[]
nfold=5
kfold = KFold(nfold,  True )
for train, test in kfold.split(X_raw_train_npy_orig):
#	print('train: %s, test: %s' % (X_raw_train_npy_orig[train].shape, X_raw_train_npy_orig[test].shape))
    X_raw_train=X_raw_train_npy_orig[train]
    X_raw_test=X_raw_train_npy_orig[test]
    y_train=np.array(y_train_npy_orig[train] , dtype=float)
    y_test=np.array(y_train_npy_orig[test], dtype=float)
    
    X_raw_train = torch.from_numpy(X_raw_train) #format torch data
    y_train = torch.from_numpy(y_train)
    X_raw_test = torch.from_numpy(X_raw_test)
    y_test = torch.from_numpy(y_test)

    X_raw_train = X_raw_train.float() #float
    X_raw_test = X_raw_test.float()
    
    #X_train = X_raw_train / 255.0 #scale
    #X_test = X_raw_test / 255.0
    X_train = X_raw_train 
    X_test = X_raw_test
    
    train_dataset = MNISTDataset(X_train, y_train) #pytorch data
    test_dataset = MNISTDataset(X_test, y_test)
    
    batch_size = 16
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
        # Define model input and output dimensions from data
    input_dim = X_train.shape[1]                  # second dimension is data dimensionality
    output_dim = len(np.unique(y_train.numpy()))  # output dim. is equal to number of unique labels

    # Instantiate multi-layer perceptron model
    mlp_model = MLP(input_dim=input_dim, output_dim=output_dim)

    # Instantiate stochastic gradient descent (SGD) optimizer
    lr = 0.05
    optimizer = torch.optim.SGD(mlp_model.parameters(), lr=lr)
    #optimizer = torch.optim.Adam(mlp_model.parameters())

    
    print('\nTraining logistic regression model...\n')# Time model training
    time_start = time.time()

    # Place model in training mode
    # .train() method affects operations such as dropout and batch normalization
    mlp_model.train()

    # Train model/iterate over epochs
    for epoch in range(n_epochs):
        # Initialize epoch metrics variables
        n_obs = 0
        loss_sum = 0
        n_correct = 0

        # Iterate through training data mini-batches
        for images_batch, labels_batch in train_loader:
            # Zero out the gradients
            optimizer.zero_grad()

            # Forward pass
            y_pred = mlp_model(images_batch)         # model predictions
            loss = loss_func(y_pred, labels_batch.long())  # loss function evaluation

            # Backward pass
            loss.backward()    # backpropagation
            optimizer.step()   # update parameters according to learning rate, gradients

            # Update epoch metrics variables
            n_batch = len(labels_batch)
            n_obs += n_batch
            loss_sum += n_batch * loss.item()
            n_correct += (y_pred.argmax(1) == labels_batch.long()).float().sum().item()

        # Calculate epoch training loss and training accuracy
        loss_train = loss_sum / n_obs
        acc_train = n_correct / n_obs

        # Display training progress
        prog_disp_freq = 1   # frequency of training progress display
        if (epoch + 1) % prog_disp_freq == 0 or epoch == 0 or epoch + 1 == n_epochs:
            print('  E%02d | train loss: %s | train acc.: %s' % 
                  (epoch + 1, '{:.4f}'.format(loss_train), '{:.4f}'.format(acc_train)))

    # Time model training
    time_end = time.time()
    print('\nMLP model training complete.\n')
    print('Time to train MLP model:  %.1f s\n' % (time_end - time_start))
    # Place model in evaluation mode
    # .eval() method affects operations such as dropout and batch normalization
    mlp_model.eval()

    # Initialize test set metrics variables
    n_obs = 0
    n_correct = 0

    # Disable gradient calculation
    with torch.no_grad():
        # Iterate through test data mini-batches
        for images_batch, labels_batch in test_loader:
            # Forward pass
            y_pred = mlp_model(images_batch)  # model predictions

            # Update test set metrics variables
            n_batch = len(labels_batch)
            n_obs += n_batch
            n_correct += (y_pred.argmax(1) == labels_batch.long()).float().sum().item()

    # Calculate test accuracy
    acc_test = n_correct / n_obs

    # Display test accuracy
    #print('\nMLP model MNIST test acc.:  %.4f\n' % (acc_test))
    testkfold.append(acc_test)



Training logistic regression model...

  E01 | train loss: 0.6914 | train acc.: 0.5769
  E02 | train loss: 0.6806 | train acc.: 0.6538
  E03 | train loss: 0.6704 | train acc.: 0.6923
  E04 | train loss: 0.6604 | train acc.: 0.7692
  E05 | train loss: 0.6537 | train acc.: 0.7308
  E06 | train loss: 0.6398 | train acc.: 0.8462
  E07 | train loss: 0.6297 | train acc.: 0.8462
  E08 | train loss: 0.6198 | train acc.: 0.8462
  E09 | train loss: 0.6100 | train acc.: 0.8462
  E10 | train loss: 0.5987 | train acc.: 0.9231
  E11 | train loss: 0.5884 | train acc.: 0.9231
  E12 | train loss: 0.5753 | train acc.: 0.9615
  E13 | train loss: 0.5636 | train acc.: 0.9615
  E14 | train loss: 0.5505 | train acc.: 0.9615
  E15 | train loss: 0.5393 | train acc.: 0.9615
  E16 | train loss: 0.5217 | train acc.: 0.9615
  E17 | train loss: 0.5052 | train acc.: 0.9615
  E18 | train loss: 0.4904 | train acc.: 1.0000
  E19 | train loss: 0.4718 | train acc.: 1.0000
  E20 | train loss: 0.4540 | train acc.: 1.0000


  E176 | train loss: 0.0050 | train acc.: 1.0000
  E177 | train loss: 0.0049 | train acc.: 1.0000
  E178 | train loss: 0.0049 | train acc.: 1.0000
  E179 | train loss: 0.0048 | train acc.: 1.0000
  E180 | train loss: 0.0048 | train acc.: 1.0000
  E181 | train loss: 0.0047 | train acc.: 1.0000
  E182 | train loss: 0.0047 | train acc.: 1.0000
  E183 | train loss: 0.0046 | train acc.: 1.0000
  E184 | train loss: 0.0046 | train acc.: 1.0000
  E185 | train loss: 0.0045 | train acc.: 1.0000
  E186 | train loss: 0.0045 | train acc.: 1.0000
  E187 | train loss: 0.0045 | train acc.: 1.0000
  E188 | train loss: 0.0044 | train acc.: 1.0000
  E189 | train loss: 0.0044 | train acc.: 1.0000
  E190 | train loss: 0.0043 | train acc.: 1.0000
  E191 | train loss: 0.0043 | train acc.: 1.0000
  E192 | train loss: 0.0042 | train acc.: 1.0000
  E193 | train loss: 0.0042 | train acc.: 1.0000
  E194 | train loss: 0.0042 | train acc.: 1.0000
  E195 | train loss: 0.0041 | train acc.: 1.0000
  E196 | train loss:

  E345 | train loss: 0.0015 | train acc.: 1.0000
  E346 | train loss: 0.0015 | train acc.: 1.0000
  E347 | train loss: 0.0015 | train acc.: 1.0000
  E348 | train loss: 0.0015 | train acc.: 1.0000
  E349 | train loss: 0.0015 | train acc.: 1.0000
  E350 | train loss: 0.0015 | train acc.: 1.0000
  E351 | train loss: 0.0015 | train acc.: 1.0000
  E352 | train loss: 0.0015 | train acc.: 1.0000
  E353 | train loss: 0.0015 | train acc.: 1.0000
  E354 | train loss: 0.0015 | train acc.: 1.0000
  E355 | train loss: 0.0015 | train acc.: 1.0000
  E356 | train loss: 0.0015 | train acc.: 1.0000
  E357 | train loss: 0.0015 | train acc.: 1.0000
  E358 | train loss: 0.0014 | train acc.: 1.0000
  E359 | train loss: 0.0014 | train acc.: 1.0000
  E360 | train loss: 0.0014 | train acc.: 1.0000
  E361 | train loss: 0.0014 | train acc.: 1.0000
  E362 | train loss: 0.0014 | train acc.: 1.0000
  E363 | train loss: 0.0014 | train acc.: 1.0000
  E364 | train loss: 0.0014 | train acc.: 1.0000
  E365 | train loss:

  E11 | train loss: 0.6311 | train acc.: 0.6154
  E12 | train loss: 0.6260 | train acc.: 0.6154
  E13 | train loss: 0.6211 | train acc.: 0.6154
  E14 | train loss: 0.6152 | train acc.: 0.6154
  E15 | train loss: 0.6094 | train acc.: 0.6154
  E16 | train loss: 0.6043 | train acc.: 0.6154
  E17 | train loss: 0.5982 | train acc.: 0.6154
  E18 | train loss: 0.5921 | train acc.: 0.6154
  E19 | train loss: 0.5863 | train acc.: 0.6154
  E20 | train loss: 0.5801 | train acc.: 0.6154
  E21 | train loss: 0.5731 | train acc.: 0.6154
  E22 | train loss: 0.5662 | train acc.: 0.6538
  E23 | train loss: 0.5591 | train acc.: 0.6923
  E24 | train loss: 0.5501 | train acc.: 0.6923
  E25 | train loss: 0.5410 | train acc.: 0.6923
  E26 | train loss: 0.5326 | train acc.: 0.6923
  E27 | train loss: 0.5226 | train acc.: 0.6923
  E28 | train loss: 0.5119 | train acc.: 0.6923
  E29 | train loss: 0.5016 | train acc.: 0.7308
  E30 | train loss: 0.4909 | train acc.: 0.7308
  E31 | train loss: 0.4802 | train acc.:

  E186 | train loss: 0.0052 | train acc.: 1.0000
  E187 | train loss: 0.0051 | train acc.: 1.0000
  E188 | train loss: 0.0050 | train acc.: 1.0000
  E189 | train loss: 0.0050 | train acc.: 1.0000
  E190 | train loss: 0.0049 | train acc.: 1.0000
  E191 | train loss: 0.0049 | train acc.: 1.0000
  E192 | train loss: 0.0048 | train acc.: 1.0000
  E193 | train loss: 0.0048 | train acc.: 1.0000
  E194 | train loss: 0.0047 | train acc.: 1.0000
  E195 | train loss: 0.0046 | train acc.: 1.0000
  E196 | train loss: 0.0046 | train acc.: 1.0000
  E197 | train loss: 0.0045 | train acc.: 1.0000
  E198 | train loss: 0.0045 | train acc.: 1.0000
  E199 | train loss: 0.0044 | train acc.: 1.0000
  E200 | train loss: 0.0044 | train acc.: 1.0000
  E201 | train loss: 0.0043 | train acc.: 1.0000
  E202 | train loss: 0.0043 | train acc.: 1.0000
  E203 | train loss: 0.0043 | train acc.: 1.0000
  E204 | train loss: 0.0042 | train acc.: 1.0000
  E205 | train loss: 0.0042 | train acc.: 1.0000
  E206 | train loss:

...

  E233 | train loss: 0.0024 | train acc.: 1.0000
  E234 | train loss: 0.0024 | train acc.: 1.0000
  E235 | train loss: 0.0024 | train acc.: 1.0000
  E236 | train loss: 0.0024 | train acc.: 1.0000
  E237 | train loss: 0.0023 | train acc.: 1.0000
  E238 | train loss: 0.0023 | train acc.: 1.0000
  E239 | train loss: 0.0023 | train acc.: 1.0000
  E240 | train loss: 0.0023 | train acc.: 1.0000
  E241 | train loss: 0.0023 | train acc.: 1.0000
  E242 | train loss: 0.0023 | train acc.: 1.0000
  E243 | train loss: 0.0022 | train acc.: 1.0000
  E244 | train loss: 0.0022 | train acc.: 1.0000
  E245 | train loss: 0.0022 | train acc.: 1.0000
  E246 | train loss: 0.0022 | train acc.: 1.0000
  E247 | train loss: 0.0022 | train acc.: 1.0000
  E248 | train loss: 0.0022 | train acc.: 1.0000
  E249 | train loss: 0.0021 | train acc.: 1.0000
  E250 | train loss: 0.0021 | train acc.: 1.0000
  E251 | train loss: 0.0021 | train acc.: 1.0000
  E252 | train loss: 0.0021 | train acc.: 1.0000
  E253 | train loss:

  E405 | train loss: 0.0010 | train acc.: 1.0000
  E406 | train loss: 0.0010 | train acc.: 1.0000
  E407 | train loss: 0.0010 | train acc.: 1.0000
  E408 | train loss: 0.0010 | train acc.: 1.0000
  E409 | train loss: 0.0010 | train acc.: 1.0000
  E410 | train loss: 0.0010 | train acc.: 1.0000
  E411 | train loss: 0.0010 | train acc.: 1.0000
  E412 | train loss: 0.0009 | train acc.: 1.0000
  E413 | train loss: 0.0009 | train acc.: 1.0000
  E414 | train loss: 0.0009 | train acc.: 1.0000
  E415 | train loss: 0.0009 | train acc.: 1.0000
  E416 | train loss: 0.0009 | train acc.: 1.0000
  E417 | train loss: 0.0009 | train acc.: 1.0000
  E418 | train loss: 0.0009 | train acc.: 1.0000
  E419 | train loss: 0.0009 | train acc.: 1.0000
  E420 | train loss: 0.0009 | train acc.: 1.0000
  E421 | train loss: 0.0009 | train acc.: 1.0000
  E422 | train loss: 0.0009 | train acc.: 1.0000
  E423 | train loss: 0.0009 | train acc.: 1.0000
  E424 | train loss: 0.0009 | train acc.: 1.0000
  E425 | train loss:

  E79 | train loss: 0.0293 | train acc.: 1.0000
  E80 | train loss: 0.0284 | train acc.: 1.0000
  E81 | train loss: 0.0275 | train acc.: 1.0000
  E82 | train loss: 0.0267 | train acc.: 1.0000
  E83 | train loss: 0.0259 | train acc.: 1.0000
  E84 | train loss: 0.0251 | train acc.: 1.0000
  E85 | train loss: 0.0244 | train acc.: 1.0000
  E86 | train loss: 0.0237 | train acc.: 1.0000
  E87 | train loss: 0.0231 | train acc.: 1.0000
  E88 | train loss: 0.0225 | train acc.: 1.0000
  E89 | train loss: 0.0218 | train acc.: 1.0000
  E90 | train loss: 0.0212 | train acc.: 1.0000
  E91 | train loss: 0.0207 | train acc.: 1.0000
  E92 | train loss: 0.0201 | train acc.: 1.0000
  E93 | train loss: 0.0195 | train acc.: 1.0000
  E94 | train loss: 0.0190 | train acc.: 1.0000
  E95 | train loss: 0.0185 | train acc.: 1.0000
  E96 | train loss: 0.0181 | train acc.: 1.0000
  E97 | train loss: 0.0177 | train acc.: 1.0000
  E98 | train loss: 0.0172 | train acc.: 1.0000
  E99 | train loss: 0.0168 | train acc.:

  E254 | train loss: 0.0023 | train acc.: 1.0000
  E255 | train loss: 0.0023 | train acc.: 1.0000
  E256 | train loss: 0.0023 | train acc.: 1.0000
  E257 | train loss: 0.0022 | train acc.: 1.0000
  E258 | train loss: 0.0022 | train acc.: 1.0000
  E259 | train loss: 0.0022 | train acc.: 1.0000
  E260 | train loss: 0.0022 | train acc.: 1.0000
  E261 | train loss: 0.0022 | train acc.: 1.0000
  E262 | train loss: 0.0022 | train acc.: 1.0000
  E263 | train loss: 0.0021 | train acc.: 1.0000
  E264 | train loss: 0.0021 | train acc.: 1.0000
  E265 | train loss: 0.0021 | train acc.: 1.0000
  E266 | train loss: 0.0021 | train acc.: 1.0000
  E267 | train loss: 0.0021 | train acc.: 1.0000
  E268 | train loss: 0.0021 | train acc.: 1.0000
  E269 | train loss: 0.0021 | train acc.: 1.0000
  E270 | train loss: 0.0020 | train acc.: 1.0000
  E271 | train loss: 0.0020 | train acc.: 1.0000
  E272 | train loss: 0.0020 | train acc.: 1.0000
  E273 | train loss: 0.0020 | train acc.: 1.0000
  E274 | train loss:

  E435 | train loss: 0.0009 | train acc.: 1.0000
  E436 | train loss: 0.0009 | train acc.: 1.0000
  E437 | train loss: 0.0009 | train acc.: 1.0000
  E438 | train loss: 0.0009 | train acc.: 1.0000
  E439 | train loss: 0.0009 | train acc.: 1.0000
  E440 | train loss: 0.0009 | train acc.: 1.0000
  E441 | train loss: 0.0009 | train acc.: 1.0000
  E442 | train loss: 0.0009 | train acc.: 1.0000
  E443 | train loss: 0.0009 | train acc.: 1.0000
  E444 | train loss: 0.0009 | train acc.: 1.0000
  E445 | train loss: 0.0009 | train acc.: 1.0000
  E446 | train loss: 0.0009 | train acc.: 1.0000
  E447 | train loss: 0.0009 | train acc.: 1.0000
  E448 | train loss: 0.0009 | train acc.: 1.0000
  E449 | train loss: 0.0009 | train acc.: 1.0000
  E450 | train loss: 0.0009 | train acc.: 1.0000
  E451 | train loss: 0.0009 | train acc.: 1.0000
  E452 | train loss: 0.0009 | train acc.: 1.0000
  E453 | train loss: 0.0008 | train acc.: 1.0000
  E454 | train loss: 0.0008 | train acc.: 1.0000
  E455 | train loss:

In [86]:
np.mean(testkfold)

0.8190476190476191

In [87]:
testkfold

[0.7142857142857143,
 1.0,
 0.7142857142857143,
 0.8333333333333334,
 0.8333333333333334]

In [88]:
#X_raw_test